This notebook illustrates how to use python to assist data collection for academic research. 

The collected data should only serve academic research purposes. 

One should never post the collected data in publicly accessible websites without explicit consent from the original website. 

One should never profit financially from the collected data. 

# Install required packages (only need to run the first time)

In [ ]:
# First: install Anaconda python: https://www.anaconda.com/products/individual

# install necessary packages
## package: selenium for navigating webpages using google chrome
# Note: also need to download chromedriver to hard drive
!pip install selenium
# install using: 
!pip3 install -U selenium
# then:
!pip3 install webdriver-manager

## package: time for pausing the script execution
!pip install time
## package: BeautifulSoup for processing web scripts
!pip install BeautifulSoup
## package: pandas for process data
!pip install pandas
## package: os for accessing operating system information
!pip install os
## package: html5lib for parsing html
!pip install html5lib

# Import required packages

In [1]:
from selenium import webdriver
# import Keys to allow for hotkeys
from selenium.webdriver.common.keys import Keys
# import ActionChains to allow for page scrolling
from selenium.webdriver.common.action_chains import ActionChains
# import expected_conditions to wait for elements to appear in web before preceeding
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# import Select to select options
from selenium.webdriver.support.ui import Select

from selenium.webdriver.chrome.service import Service
# install using: 
#   pip3 install -U selenium
# then:
#   pip3 install webdriver-manager
from webdriver_manager.chrome import ChromeDriverManager
import time

from time import sleep

from bs4 import BeautifulSoup as soup

import pandas as pd

import os.path

# Python selenium with chromedriver

We will use python selenium to control a chromedriver application to navigate websites.

The chromedriver is an open source software that needs to be downloaded to the hard drive. 

After downloading the chromedriver to local disk, change the path and run:

In [2]:
# start a chrome browser
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

[WDM] - Downloading: 100%|██████████| 6.79M/6.79M [00:00<00:00, 20.6MB/s]


set url and load url

In [3]:
# set url (as a text string)
url = "https://www.thedialogue.org/map_list/"
# load url
driver.get(url)
# notes: 
## driver is an object, the webdriver controlled by selenium
## get is a method (think of it as a function) defined in the selenium package

# Task 1: collect information from a table

In this task, we will manually navigate to a table and then use bs4 and pandas to collect the information in a html table.

From this site, we manually click: "China-Latin America Finance Database", and then click the first country on the list "Venezuela".

In [14]:
driver.find_element(By.XPATH, value="//div[@class='link policy']").click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='link policy']"}
  (Session info: chrome=109.0.5414.120)
Stacktrace:
Backtrace:
	(No symbol) [0x00396643]
	(No symbol) [0x0032BE21]
	(No symbol) [0x0022DA9D]
	(No symbol) [0x00261342]
	(No symbol) [0x0026147B]
	(No symbol) [0x00298DC2]
	(No symbol) [0x0027FDC4]
	(No symbol) [0x00296B09]
	(No symbol) [0x0027FB76]
	(No symbol) [0x002549C1]
	(No symbol) [0x00255E5D]
	GetHandleVerifier [0x0060A142+2497106]
	GetHandleVerifier [0x006385D3+2686691]
	GetHandleVerifier [0x0063BB9C+2700460]
	GetHandleVerifier [0x00443B10+635936]
	(No symbol) [0x00334A1F]
	(No symbol) [0x0033A418]
	(No symbol) [0x0033A505]
	(No symbol) [0x0034508B]
	BaseThreadInitThunk [0x75FA00F9+25]
	RtlGetAppContainerNamedObjectPath [0x77157BBE+286]
	RtlGetAppContainerNamedObjectPath [0x77157B8E+238]


This website has `iframe` that contains the content source code. We need to first ask the driver to switch to the frame in order to access elements inside the frame. 

After searching `//iframe` in the browser source code, we found two iframes, the main content iframe is the first one. We will ask driver to switch to this frame.

In [15]:
driver.switch_to.frame(driver.find_elements(By.XPATH, "//iframe")[0])

Now the driver will be able to click on the link.

In [16]:
driver.find_element(By.XPATH, value="//div[@class='link policy']").click()

Click on the first country in the country list. We skip the first div because it is a divider.

In [17]:
# find all elements in the country list, then click on the second one (first one is an empty divider)
driver.find_elements(By.XPATH, value="//div[@class='NestedList Policy active']/div")[1].click()

Right click on the table, and choose "inspect". 

In the "Elements" pane, we see that the table is contained under a `<table>` tag, with attribute `class="investmentTable"`.

We would like to check if the table is unique on the webpage. For that, click in the "Elements" pane, and then hit "Cmd + F" on mac, or "Ctr + F" on PC. In the search bar, type xpath selector `//table` to see how many tables can be found. We found only one.

The python pandas package contains a function that parses data from html table very efficiently. We will take this approach.

In [24]:
# read (the first and only) table into a list
table_df = pd.read_html(driver.page_source)
# convert list to dataframe
table_df = pd.DataFrame(table_df[0])
# inspect dataframe
table_df

,Date,Types,Purpose,Lender,Amount
0,November 2007,Energy,Joint Fund - Tranche A,CDB,$4.0B
1,April 2009,Energy,Joint Fund - Tranche B,CDB,$4.0B
2,December 2009,Mining,Mining project credit,CDB,$1.0B
3,December 2009,Energy,Not specified,Ex-Im Bank,$500M
4,August 2010,Energy,Joint Fund - Long-Term Facility,CDB,$20.3B
5,June 2011,Energy,Joint Fund - Tranche A renewal,CDB,$4.0B
6,November 2011,Energy,Abreu e Lima refinery,CDB,$1.5B
7,February 2012,Energy,Purchase of oil-related products,CDB,$500M
8,August 2012,Energy,Joint Fund - Tranche B renewal,CDB,$4.0B
9,June 2013,Energy,Sinovensa production in Orinoco,CDB,$4.0B


In [33]:
# add a column for the country name
table_df['Country'] = "Venezuela"
# save dataframe to csv
table_df.to_csv("/Users/lhe/Downloads/finance_Venezuela.csv", index = False)
# inspect dataframe
table_df

,Date,Types,Purpose,Lender,Amount,Country
0,November 2007,Energy,Joint Fund - Tranche A,CDB,$4.0B,Venezuela
1,April 2009,Energy,Joint Fund - Tranche B,CDB,$4.0B,Venezuela
2,December 2009,Mining,Mining project credit,CDB,$1.0B,Venezuela
3,December 2009,Energy,Not specified,Ex-Im Bank,$500M,Venezuela
4,August 2010,Energy,Joint Fund - Long-Term Facility,CDB,$20.3B,Venezuela
5,June 2011,Energy,Joint Fund - Tranche A renewal,CDB,$4.0B,Venezuela
6,November 2011,Energy,Abreu e Lima refinery,CDB,$1.5B,Venezuela
7,February 2012,Energy,Purchase of oil-related products,CDB,$500M,Venezuela
8,August 2012,Energy,Joint Fund - Tranche B renewal,CDB,$4.0B,Venezuela
9,June 2013,Energy,Sinovensa production in Orinoco,CDB,$4.0B,Venezuela


Browser back to previous page, returning to the list of countries.

In [37]:
# driver back
driver.back()

# Task 2: Loop through the list of countries

We can keep clicking on each country by hand, or change the index of country div in the previous code, and repeat the process to manually save the table for each country. But is there a more automated way?

One possibility is to program the driver to loop through each country on the list. 

## Preliminary: for loop through a range

In python, there are many ways to construct a loop. One of the most basic loop is a for loop through a range. 

A for loop through the range [1,10] will start at 1 but ends at 9, not including the high-end of the interval. 

In [ ]:
# loop through a range
for i in range(0,10):
    print(i)

0
1
2
3
4
5
6
7
8
9


Note that, unlike stata, the loop is not indicated by braces `{}`, but by a colon `:` and indentation with four spaces. All indented code following the `:` is part of the loop.

In [ ]:
# loop through a range; all indented parts will be executed in the loop; unindented part will be executed after the loop
for i in range(0,10):
    print(f"The current number is {i}")
    print(f"The next number will be {i+1}")
    
    print("------------------------")
    
print(f"The final number is 9, not 10")

The current number is 0
The next number will be 1
------------------------
The current number is 1
The next number will be 2
------------------------
The current number is 2
The next number will be 3
------------------------
The current number is 3
The next number will be 4
------------------------
The current number is 4
The next number will be 5
------------------------
The current number is 5
The next number will be 6
------------------------
The current number is 6
The next number will be 7
------------------------
The current number is 7
The next number will be 8
------------------------
The current number is 8
The next number will be 9
------------------------
The current number is 9
The next number will be 10
------------------------
The final number is 9, not 10


## Back on the task

### Key elements of the loop

What is the process to repeat? 
1. click on country $i$, for $i=1,2,3,...$
2. extract the table from the new page
3. add country name to table
4. save the table to file (needs country name for file name)
5. back to country list page
6. click on country $i+1$

Recall that the driver returns a list of countries from the element search `driver.find_elements(By.XPATH, value="//div[@class='NestedList Policy active']/div")`. We simply chose to click on the first country (second `div`).

We can ask python to loop through each element in this list, starting from the second one.

In [8]:
# set url (as a text string)
url = "https://www.thedialogue.org/map_list/"
# load url
driver.get(url)

# wait for the page to load
sleep(3)

# make sure driver swtiches to the content iframe
driver.switch_to.frame(driver.find_elements(By.XPATH, "//iframe")[0])

# click to finance list
driver.find_element(By.XPATH, value="//div[@class='link policy']").click()

In [15]:
# put all elements in the country list into a list, only take the second one onwards
country_element_list = driver.find_elements(By.XPATH, value="//div[@class='NestedList Policy active']/div")[1:]
# count number of countries in the list
len(country_element_list)

19

In [18]:
# loop through all elements found previously in news_elements
for i in range(1, len(country_element_list) + 1):
    print(i)
    ## navigate from list to table for each country
    try: 
        # make sure driver swtiches to the content iframe
        driver.switch_to.frame(driver.find_elements(By.XPATH, "//iframe")[0])
    except:
        pass
    
    # find all countries in the list, then click on the ith one
    driver.find_elements(By.XPATH, value="//div[@class='NestedList Policy active']/div")[i].click()
    # pause for two seconds for the page to load
    sleep(2)
    
    ## extract the table
    # read (the first and only) table into a list
    table_df = pd.read_html(driver.page_source)
    # convert list to dataframe
    table_df = pd.DataFrame(table_df[0])
    
    ## add country name and save file
    # extract country name
    country_name = driver.find_element(By.CSS_SELECTOR, value="div[class^='countryName']").text
    # add a column for the country name
    table_df['Country'] = country_name
    # save dataframe to csv
    table_df.to_csv(f"/Users/lhe/Downloads/finance_{country_name}.csv", index = False)
    
    ## navigate to the list page
    driver.back()
    # pause for two seconds for the page to load
    sleep(2)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [19]:
# set url (as a text string)
url = "https://www.thedialogue.org/map_list/"
# load url
driver.get(url)

# wait for the page to load
sleep(3)

# make sure driver swtiches to the content iframe
driver.switch_to.frame(driver.find_elements(By.XPATH, "//iframe")[0])

# click to finance list
driver.find_element(By.XPATH, value="//div[@class='link commercial']").click()

In [21]:
# put all elements in the country list into a list, only take the second one onwards
country_element_list = driver.find_elements(By.XPATH, value="//div[@class='NestedList Commercial active']/div")[1:]
# count number of countries in the list
len(country_element_list)

9

In [22]:
# loop through all elements found previously in news_elements
for i in range(1, len(country_element_list) + 1):
    print(i)
    ## navigate from list to table for each country
    try: 
        # make sure driver swtiches to the content iframe
        driver.switch_to.frame(driver.find_elements(By.XPATH, "//iframe")[0])
    except:
        pass
    
    # find all countries in the list, then click on the ith one
    driver.find_elements(By.XPATH, value="//div[@class='NestedList Commercial active']/div")[i].click()
    # pause for two seconds for the page to load
    sleep(2)
    
    ## extract the table
    # read (the first and only) table into a list
    table_df = pd.read_html(driver.page_source)
    # convert list to dataframe
    table_df = pd.DataFrame(table_df[0])
    
    ## add country name and save file
    # extract country name
    country_name = driver.find_element(By.CSS_SELECTOR, value="div[class^='countryName']").text
    # add a column for the country name
    table_df['Country'] = country_name
    # save dataframe to csv
    table_df.to_csv(f"/Users/lhe/Downloads/commercial_{country_name}.csv", index = False)
    
    ## navigate to the list page
    driver.back()
    # pause for two seconds for the page to load
    sleep(2)

1
2
3
4
5
6
7
8
9
